<a href="https://colab.research.google.com/github/HARDIK218/Projects/blob/main/StockNews_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.2 MB/s eta 0:00:00


In [2]:
from transformers import PegasusTokenizer,PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

In [3]:
!pip install sentencepiece
import sentencepiece

In [4]:
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at human-centered-summarization/financial-summarization-pegasus and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
url = "https://www.livemint.com/news/india/rbi-governor-shaktikanta-das-receives-award-for-a-ranking-in-global-finance-central-banker-report-cards-2023-11697303581727.html"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')

In [6]:
paragraphs

[<p>      Reserve Bank of India (RBI) Governor Shaktikanta Das received the Award for being ranked ‘A+’ in the Global Finance Central Banker Report Cards 2023 on Saturday in Marakkesh city of Morocco. The award recognizes those central bank governors whose strategies outperformed their peers through originality, creativity, and tenacity. RBI shared the news on the official handle on X with the picture of the RBI Governor taking the award.</p>,
 <p>   The awards were announced in the month of September and along with <a class="manualbacklink" href="https://www.livemint.com/news/india/rbi-governor-shaktikanta-das-rated-a-in-global-finance-central-banker-report-2023-11693616524045.html" target="_blank">RBI Governor Shaktikanta Das</a>, two other central bankers- Thomas J. Jordan of Switzerland and Nguyen Thi Hong of Vietnam earned an ‘A+’ grade in the Global Finance Central Banker Report Cards 2023.</p>,
 <p>   "The Bank Governors who earned an ‘A+’ grade in the Global Finance Central Ban

In [7]:
text = [paragraph.text for paragraph in paragraphs]
words = ' '.join(text).split(' ')[:400]
ARTICLE = ' '.join(words)


In [8]:
len(words)

400

In [9]:
input_ids = tokenizer.encode(ARTICLE,return_tensors='pt')
output = model.generate(input_ids,max_length=55,num_beams=5,early_stopping=True)
summary = tokenizer.decode(output[0],skip_special_tokens=True)

In [10]:
summary

'Shaktikanta Das ranked ‘A+’ in Global Finance Central Banker Report Cards. Jordan, Hong also ranked ‘A+’ in the report cards'

##Building a news pipeline

In [11]:
monitored_tickers = ['GME', 'TSLA', 'BTC']

In [12]:
#search for stock news using google and yahoo financs
def search_for_stock_news_urls(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs

In [13]:
raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls


{'GME': ['/?sa=X&ved=0ahUKEwjW1tnFv_eBAxXJqpUCHbNhD-gQOwgC',
  '/search?q=yahoo+finance+GME&tbm=nws&sca_esv=573559708&ie=UTF-8&gbv=1&sei=LZArZZaMD8nV1sQPs8O9wA4',
  '/search?q=yahoo+finance+GME&sca_esv=573559708&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwjW1tnFv_eBAxXJqpUCHbNhD-gQ_AUIBSgA',
  '/search?q=yahoo+finance+GME&sca_esv=573559708&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwjW1tnFv_eBAxXJqpUCHbNhD-gQ_AUIBygC',
  '/search?q=yahoo+finance+GME&sca_esv=573559708&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwjW1tnFv_eBAxXJqpUCHbNhD-gQ_AUICCgD',
  'https://maps.google.com/maps?q=yahoo+finance+GME&um=1&ie=UTF-8&sa=X&ved=0ahUKEwjW1tnFv_eBAxXJqpUCHbNhD-gQ_AUICSgE',
  '/search?q=yahoo+finance+GME&sca_esv=573559708&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwjW1tnFv_eBAxXJqpUCHbNhD-gQ_AUICigF',
  '/search?q=yahoo+finance+GME&sca_esv=573559708&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwjW1tnFv_eBAxXJqpUCHbNhD-gQ_AUICygG',
  '/advanced_search',
  '/search?q=yahoo+finance+GME&sca_esv=57355970

In [14]:
raw_urls['GME'][0]

'/?sa=X&ved=0ahUKEwjW1tnFv_eBAxXJqpUCHbNhD-gQOwgC'

In [15]:
#strip out unwanted urls
import re
exclude_list = ['maps','policies','prefrences','accounts','support']

In [16]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))


In [17]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_tickers}
cleaned_urls

{'GME': ['https://www.wsj.com/articles/the-lessons-of-dumb-money-gamestop-stock-craze-movie-daa0d0bf',
  'https://time.com/6312307/gamestop-meme-stocks-dumb-money/',
  'https://www.theguardian.com/film/2023/sep/09/dumb-money-movie-review-gamestop-paul-dano-seth-rogan',
  'https://www.cnbc.com/2023/09/15/heres-where-amc-and-gamestop-are-now-as-dumb-money-hits-theaters.html',
  'https://www.investopedia.com/keith-gill-roaring-kitty-8303143',
  'https://www.vox.com/money/2023/9/15/23873474/dumb-money-gamestop-stock-keith-gill-melvin-capital-review',
  'https://investorplace.com/2023/09/7-meme-stocks-to-sell-in-september-before-they-crash-burn/',
  'https://www.cnbc.com/2023/09/28/gamestop-names-ryan-cohen-as-ceo-effective-immediately.html',
  'https://www.barrons.com/articles/gamestop-stock-price-ryan-cohen-aa31dd8b',
  'https://fortune.com/2023/09/28/gamestop-names-ryan-cohen-ceo-meme-stock-chewy-founder/',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BGME%26tbm%3Dnws%26pccc%3D1

In [18]:
## SEARCH AND SCRAPE CLEANED URLS
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [19]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'GME': ['WSJ Membership Customer Service Tools & Features Ads More Dow Jones Products',
  'In early 2021, millions of ordinary people across the United States began buying shares of the declining video game store GameStop in increments of tens, hundreds or thousands of dollars. Their investments were mocked by financial experts, who cited the store’s poor underlying metrics. But the masses’ collective action proved shockingly powerful: they pushed GameStop’s price from under $3 to as high as $483 in late January 2021, causing the hedge funds that had bet against it to lose billions of dollars.  Nearly three years later, the new movie Dumb Money attempts to dramatize and make sense of the strange episode. Paul Dano stars as Keith Gill, a trader and livestreamer whose full-throated support of the stock on the subreddit community Wall Street Bets catalyzed a movement around “meme stocks”: assets that surged in price fueled by social media enthusiasm. Seth Rogen, Pete Davidson and America

In [20]:
def summarize(articles):
    summaries = []
    for article in articles:
        input_ids = tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [21]:
summaries = {ticker:summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

{'GME': ['Rough Rough cut (no reporter narration)',
  'Paul Dano, Seth Rogen and America Ferrera star in movie. Retail investors, Wall Street power plays central role',
  'Paul Dano, Seth Rogen star in ‘Dumb Money’. It’s an ensemble piece, not just a financial drama',
  'Find the best credit cards, loans, insurance and more in SELECT.',
  'All photographs subject to copyright.',
  'Back in early 2021, a ragtag group of everyday traders took on Wall Street. But two and a half years later, the story is a whole lot messier',
  'Peloton, Mullen Automotive among meme stocks to sell. These stocks are flashing red, offering little upside ahead',
  'Find the best credit cards, loans, insurance and more in SELECT.',
  'Have you tried going to Newsround?',
  'Cohen already serves as chairman of the board. Video game retailer has been without a CEO since June',
  'All images are copyrighted.',
  'We We use cookies to improve the experience for users.'],
 'TSLA': ['Year-to-date EV sales through Se

##Adding Sentiment Analysis

In [3]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
scores = {ticker:sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

In [ ]:
print(summaries['GME'][3], scores['GME'][3]['label'], scores['GME'][3]['score'])

##Exporting to CSV file

In [ ]:
summaries

In [ ]:
scores

In [ ]:
cleaned_urls

In [ ]:
def create_output_array(summaries,scores,urls):
  output = []
  for ticker in monitored_tickers:
    for counter in range(len(summaries[ticker])):
      output_this = [
          ticker,
          summaries[ticker][counter],
          scores[ticker][counter]['label']
          scores[ticker][counter]['score']
          url[ticker][counter]

      ]
      output.append(output_this)
    return output